
### In this notebook, the data is loaded into pandas dataframes, and all initial preprocessing steps are done. Because of company regulations, the RTL data and file names (Videoland) are excluded.

In [0]:
import numpy as np
import pandas as pd

In [0]:
# loading in the open source MPST dataset after downloading the CSV from Kaggle.

mpst_df = pd.read_csv('mpst_full_data.csv')
mpst_df.head()
len(mpst_df)

14828

In [0]:
# nulls and duplicates check:

columns_to_check = [
    'imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source'
]

for col in columns_to_check:
    unique_count = mpst_df[col].nunique()
    print(f"Total unique entries in {col}: {unique_count}")

for col in columns_to_check:
    nulls_count = mpst_df[col].isna().sum()
    print(f"Total null entries in {col}: {nulls_count}")

Total unique entries in imdb_id: 14828
Total unique entries in title: 13757
Total unique entries in plot_synopsis: 13848
Total unique entries in tags: 5604
Total unique entries in split: 3
Total unique entries in synopsis_source: 2
Total null entries in imdb_id: 0
Total null entries in title: 0
Total null entries in plot_synopsis: 0
Total null entries in tags: 0
Total null entries in split: 0
Total null entries in synopsis_source: 0


In [0]:
# Print the collection of unique tags used:

unique_tags = set(tag.strip() for tags in mpst_df['tags'] for tag in tags.split(','))

# Convert to list or print the set directly
print(unique_tags)
len(unique_tags)

{'sentimental', 'alternate history', 'psychological', 'absurd', 'philosophical', 'pornographic', 'murder', 'violence', 'adult comedy', 'entertaining', 'neo noir', 'good versus evil', 'sci-fi', 'claustrophobic', 'suspenseful', 'historical fiction', 'clever', 'paranormal', 'cute', 'action', 'blaxploitation', 'home movie', 'sadist', 'atmospheric', 'depressing', 'christian film', 'magical realism', 'avant garde', 'feel-good', 'cruelty', 'flashback', 'thought-provoking', 'prank', 'anti war', 'stupid', 'boring', 'melodrama', 'alternate reality', 'haunting', 'storytelling', 'queer', 'revenge', 'psychedelic', 'gothic', 'comedy', 'autobiographical', 'insanity', 'dark', 'western', 'bleak', 'plot twist', 'horror', 'fantasy', 'brainwashing', 'mystery', 'historical', 'suicidal', 'allegory', 'realism', 'comic', 'romantic', 'grindhouse film', 'non fiction', 'cult', 'satire', 'whimsical', 'inspiring', 'tragedy', 'humor', 'intrigue', 'dramatic'}


71

In [0]:
empty_tags_count = mpst_df['tags'].apply(lambda x: x == '' or pd.isna(x)).sum()

print(f"Empty cells in 'tags' column: {empty_tags_count}")

Empty cells in 'tags' column: 0


In [0]:
mpst_df = mpst_df[(mpst_df['tags'].notna()) & (mpst_df['tags'] != '')]
len(mpst_df)

14828


### Querying Wikidata for Awards data

In [0]:
#creating chunks of imdb_ids from MPST data sized 500:

mpst_df['imdb_id'].to_csv("imdb_ids_mpst", index=False, header=False)

In [0]:
with open("imdb_ids_mpst", "r") as file:
    imdb_ids = [line.strip() for line in file.readlines()]

chunk_size = 500

# split the IMDb IDs into chunks of 500
imdb_chunks = [imdb_ids[i:i + chunk_size] for i in range(0, len(imdb_ids), chunk_size)]


#### Use the following query in Wikidata Query Service, and fill in the imdb_ids from the generated chunk files to retrieve the award data:

Link: https://query.wikidata.org/


In [0]:
# %sql

#     SELECT ?imdb_id ?title ?award_label WHERE {
#       ?film wdt:P31 wd:Q11424;  # P31 = instance of, Q11424 = film
#             wdt:P345 ?imdb_id;   # P345 = IMDb ID
#             wdt:P166 ?award.     # P166 = award received
#       ?film rdfs:label ?title.
#       ?award rdfs:label ?award_label.
#       FILTER(?imdb_id IN ("....."))
#       FILTER(LANG(?title) = "en")
#       FILTER(LANG(?award_label) = "en")
#     }

#### Download the response files from Wikidata Query Service, and upload to the notebooks.

In [0]:
wiki_awards = pd.read_csv('merged_awards.csv')
wiki_awards

,imdb_id,title,award_label
0,tt0070735,The Sting,Academy Award for Best Production Design
1,tt0070735,The Sting,Academy Award for Best Film Editing
2,tt0070735,The Sting,Academy Award for Best Director
3,tt0070735,The Sting,"Academy Award for Best Writing, Original Scree..."
4,tt0070735,The Sting,Academy Award for Best Picture
...,...,...,...
4826,tt3741834,Lion,AACTA Award for Best Actor in a Leading Role
4827,tt3741834,Lion,AACTA Award for Best Cinematography
4828,tt3741834,Lion,AACTA Award for Best Costume Design
4829,tt3741834,Lion,AACTA Award for Best Direction


In [0]:
wiki_awards = wiki_awards.groupby(['imdb_id', 'title'], as_index=False)['award_label'].apply(lambda x: ', '.join(x)).reset_index(drop=True)
wiki_awards = wiki_awards.drop(index=0).reset_index(drop=True)
wiki_awards

,imdb_id,title,award_label
0,tt0019729,The Broadway Melody,Academy Award for Best Picture
1,tt0019788,Coquette,Academy Award for Best Actress
2,tt0020629,All Quiet on the Western Front,"Academy Award for Best Picture, Academy Award ..."
3,tt0020815,The Dawn Patrol,Academy Award for Best Story
4,tt0021156,Morocco,National Board of Review: Top Ten Films
...,...,...,...
1768,tt4730986,Divines,César Award for Best Supporting Actress
1769,tt4975722,Moonlight,"Academy Award for Best Picture, Academy Award ..."
1770,tt5013056,Dunkirk,"Academy Award for Best Film Editing, Academy A..."
1771,tt5052448,Get Out,"Academy Award for Best Writing, Original Scree..."


In [0]:

#spliting each award list by commas and unpack the values
all_awards = wiki_awards['award_label'].str.split(',').explode().str.strip()

#get the unique awards
unique_awards = all_awards.unique()

#sort the awards (optional)
unique_awards_sorted = sorted(unique_awards)

wiki_awards_names = []

for award in unique_awards_sorted:
    wiki_awards_names.append(award)

wiki_awards_names
len(wiki_awards_names)

842


Now that we have the wiki_awards, we merge them into the MPST dataset, and for titles with no prize, we fill in: No award, since this can also hold generative / predictive information.

In [0]:
merged_df = pd.merge(mpst_df, wiki_awards, on='imdb_id', how='left')

merged_df['award_label'].fillna('No award', inplace=True)

merged_df = merged_df[['imdb_id', 'title_x', 'plot_synopsis', 'tags', 'split', 'synopsis_source', 'award_label']]

merged_df.rename(columns={'title_x': 'title'}, inplace=True)

In [0]:
#split each entry in 'tags' column by comma and expand into a flat list
unique_tags = set(tag.strip() for tags in merged_df['tags'] for tag in tags.split(','))

print(unique_tags)

{'sentimental', 'alternate history', 'psychological', 'absurd', 'philosophical', 'pornographic', 'murder', 'violence', 'adult comedy', 'entertaining', 'neo noir', 'good versus evil', 'sci-fi', 'claustrophobic', 'suspenseful', 'historical fiction', 'clever', 'paranormal', 'cute', 'action', 'blaxploitation', 'home movie', 'sadist', 'atmospheric', 'depressing', 'christian film', 'magical realism', 'avant garde', 'feel-good', 'cruelty', 'flashback', 'thought-provoking', 'prank', 'anti war', 'stupid', 'boring', 'melodrama', 'alternate reality', 'haunting', 'storytelling', 'queer', 'revenge', 'psychedelic', 'gothic', 'comedy', 'autobiographical', 'insanity', 'dark', 'western', 'bleak', 'plot twist', 'horror', 'fantasy', 'brainwashing', 'mystery', 'historical', 'suicidal', 'allegory', 'realism', 'comic', 'romantic', 'grindhouse film', 'non fiction', 'cult', 'satire', 'whimsical', 'inspiring', 'tragedy', 'humor', 'intrigue', 'dramatic'}


In [0]:
merged_df['tags'] = merged_df['tags'].str.split(',')

In [0]:
merged_df.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,award_label
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"[cult, horror, gothic, murder, atmospheric]",train,imdb,No award
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",[violence],train,imdb,No award
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",[romantic],test,imdb,No award
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","[inspiring, romantic, stupid, feel-good]",train,imdb,No award
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","[cruelty, murder, dramatic, cult, violence...",val,imdb,No award


In [0]:
merged_df.to_csv("/dbfs/tmp/train_test_df.csv", index=False)

In [0]:
merged_df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,award_label
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"[cult, horror, gothic, murder, atmospheric]",train,imdb,No award
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",[violence],train,imdb,No award
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",[romantic],test,imdb,No award
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","[inspiring, romantic, stupid, feel-good]",train,imdb,No award
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","[cruelty, murder, dramatic, cult, violence...",val,imdb,No award
...,...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","[comedy, murder]",test,wikipedia,No award
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","[good versus evil, violence]",train,wikipedia,No award
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,[anti war],train,wikipedia,No award
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,[murder],test,wikipedia,No award
